# Module 4: Building Durable AI Agents with the Agentic Loop Pattern

## Introduction: The Challenge of Production AI Agents

Building AI agents that work reliably in production is challenging. As the Temporal tutorial explains:

> "AI agents are complex distributed systems that face challenges like network failures, long-running workflows, and external service interruptions."

In this module, we'll explore how Temporal transforms unreliable AI agents into robust, self-healing systems through the **agentic loop pattern**.

### What You'll Learn
- 🔄 The agentic loop pattern for autonomous AI agents
- 🛡️ How Temporal provides durability and fault tolerance
- 🔧 Dynamic tool execution with Activities
- 📊 Full observability into agent decision-making
- 🚀 Building production-ready AI systems

## Understanding the Agentic Loop Pattern

The agentic loop is a powerful pattern where an AI agent autonomously:

1. **Observes** the current context and state
2. **Decides** what action to take using an LLM
3. **Acts** by executing the selected tool
4. **Updates** its context with results
5. **Repeats** until the goal is achieved

This creates a dynamic, goal-driven system that can adapt to different scenarios and handle complex, multi-step tasks.

### Why Temporal?

Traditional AI agents suffer from:
- ❌ Lost progress when LLM calls fail
- ❌ No visibility into decision-making
- ❌ Expensive re-execution after failures
- ❌ Complex retry logic scattered throughout code

Temporal provides:
- ✅ **Automatic failure handling** with configurable retry policies
- ✅ **State persistence** across interruptions
- ✅ **Full observability** into agent workflows
- ✅ **Guaranteed completion** even with crashes

## Step 1: Setup and Imports

Let's start by importing our modules and understanding the structure.

In [ ]:
# Add parent directory to path so we can import our modules
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

# Import our modules
from models import AgentGoal, BookingResult, ToolDefinition, ToolArgument
from tools import AVAILABLE_TOOLS, search_flights, check_seat_availability, calculate_total_cost, book_flight, send_confirmation
from activities import agent_validate_prompt, ai_select_tool_with_params

# Standard imports
import asyncio
import json
from datetime import timedelta
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ Modules imported successfully!")
print(f"📦 Available tools: {list(AVAILABLE_TOOLS.keys())}")

## Step 2: Exploring Our Data Models

Our agent uses structured data models to define goals, tools, and results. Let's examine them:

In [ ]:
# Let's look at what makes up an AgentGoal
from dataclasses import fields

print("📋 AgentGoal fields:")
for field in fields(AgentGoal):
    print(f"  - {field.name}: {field.type}")

print("\n🔧 ToolDefinition fields:")
for field in fields(ToolDefinition):
    print(f"  - {field.name}: {field.type}")

print("\n📊 BookingResult fields:")
for field in fields(BookingResult):
    print(f"  - {field.name}: {field.type}")

## Step 3: Understanding Available Tools

Our travel booking agent has access to several tools. Let's explore them:

In [ ]:
# Examine the available tools and their definitions
print("🛠️ Available Tools for the Agent:\n")

for tool_name, tool_def in AVAILABLE_TOOLS.items():
    print(f"📌 {tool_name}:")
    print(f"   Description: {tool_def.description}")
    print(f"   Arguments:")
    for arg in tool_def.arguments:
        print(f"     • {arg.name} ({arg.type}): {arg.description}")
    print()

## Step 4: Testing Tool Execution

Let's test executing some tools directly to see how they work:

In [ ]:
# Test the search_flights tool
print("✈️ Testing search_flights tool:\n")

search_params = {
    "origin": "NYC",
    "destination": "London",
    "date": "tomorrow"
}

result = await search_flights(search_params)
print(result)
print("\n" + "="*60)

In [ ]:
# Test the check_seat_availability tool
print("💺 Testing check_seat_availability tool:\n")

availability_params = {"flight_id": "UA456"}
result = await check_seat_availability(availability_params)
print(result)
print("\n" + "="*60)

## Step 5: Understanding the AI Decision-Making Activities

Our agent uses two key Activities for AI decision-making:

1. **`agent_validate_prompt`** - Validates that the user's request aligns with the agent's capabilities
2. **`ai_select_tool_with_params`** - AI selects which tool to use and extracts parameters

Let's see how they work:

In [ ]:
# Create a sample agent goal
sample_goal = AgentGoal(
    agent_name="Travel Booking Assistant",
    description="Book a flight from NYC to London for tomorrow",
    tools=list(AVAILABLE_TOOLS.values()),
    llm_model="openai/gpt-4o-mini",
    llm_api_key="demo-key",  # Using mock for demo
    starter_prompt="You are an expert travel agent.",
    example_conversation_history="User: I need a flight\nAgent: I'll help you find the perfect flight."
)

print("🎯 Created Agent Goal:")
print(f"  Name: {sample_goal.agent_name}")
print(f"  Task: {sample_goal.description}")
print(f"  Tools: {len(sample_goal.tools)} available")

In [ ]:
# Mock the LLM response for demonstration
import activities

# Override with mock for demo
def mock_llm_completion(model, messages, temperature, api_key):
    """Mock LLM for demonstration"""
    class MockResponse:
        class Choice:
            class Message:
                content = "YES"  # For validation
            message = Message()
        choices = [Choice()]
    return MockResponse()

# Temporarily replace for demo
original_completion = None
try:
    from litellm import completion
    original_completion = completion
except ImportError:
    pass

# Test validation
print("🔍 Testing prompt validation:\n")
is_valid = await agent_validate_prompt(
    sample_goal,
    "Book a flight from NYC to London",
    sample_goal.llm_model,
    sample_goal.llm_api_key
)
print(f"Validation result: {'✅ VALID' if is_valid else '❌ INVALID'}")

## Step 6: Examining the Workflow Implementation

Now let's look at how the agentic workflow orchestrates everything. The workflow is defined in `workflow.py`:

In [ ]:
# Let's read and display key parts of the workflow
with open('workflow.py', 'r') as f:
    workflow_code = f.read()

# Find the run method
import re
run_method = re.search(r'(\s+async def run.*?)(?=\n\s{0,4}[a-zA-Z@]|$)', workflow_code, re.DOTALL)
if run_method:
    print("📜 AgenticWorkflow.run method structure:\n")
    lines = run_method.group(1).split('\n')[:30]  # First 30 lines
    for line in lines:
        print(line)

## Step 7: Understanding the Agentic Loop Flow

The agentic loop follows this pattern:

1. **Validate** the request matches agent capabilities
2. **Select** the next tool using AI
3. **Execute** the selected tool as an Activity
4. **Update** context with results
5. **Repeat** until AI returns "DONE"

Let's visualize this:

In [ ]:
import base64
from IPython.display import Image, display

def render_mermaid(graph_definition):
    """Render a Mermaid diagram"""
    graph_bytes = graph_definition.encode("ascii")
    base64_bytes = base64.b64encode(graph_bytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

diagram = """
graph TD
    A[Start Workflow] --> B[Validate Request]
    B --> C{Valid?}
    C -->|No| D[Return Error]
    C -->|Yes| E[Start Loop]
    E --> F[AI Selects Tool]
    F --> G{Tool = DONE?}
    G -->|Yes| H[Return Success]
    G -->|No| I{Tool Valid?}
    I -->|No| J[Add Error to Context]
    J --> F
    I -->|Yes| K[Execute Tool Activity]
    K --> L[Update Context]
    L --> M{Max Iterations?}
    M -->|No| F
    M -->|Yes| H
    
    style A fill:#e1f5fe
    style F fill:#fff3e0
    style K fill:#e8f5e9
    style H fill:#c8e6c9
"""

render_mermaid(diagram)
print("\n📊 This shows the complete agentic loop with Temporal's durability")

## Step 8: Simulating the Agentic Loop

Let's simulate how the agentic loop works without running the full Temporal workflow:

In [ ]:
async def simulate_agentic_loop():
    """Simulate the agentic loop execution"""
    
    print("🚀 SIMULATING AGENTIC LOOP EXECUTION\n")
    print("="*60)
    
    context = ""
    steps_taken = []
    
    # Mock decision function
    def get_mock_decision(context):
        if not context:
            return {"tool": "search_flights", 
                   "parameters": {"origin": "NYC", "destination": "London", "date": "tomorrow"}}
        elif "Found 3 flights" in context and "Seat availability" not in context:
            return {"tool": "check_seat_availability",
                   "parameters": {"flight_id": "UA456"}}
        elif "Seat availability" in context and "Cost breakdown" not in context:
            return {"tool": "calculate_total_cost",
                   "parameters": {"flight_id": "UA456", "passengers": 1}}
        elif "Cost breakdown" in context and "booked successfully" not in context:
            return {"tool": "book_flight",
                   "parameters": {"flight_id": "UA456", "seat_class": "economy"}}
        elif "booked successfully" in context and "Confirmation email" not in context:
            return {"tool": "send_confirmation",
                   "parameters": {"confirmation_number": "CONF-UA456-1234", "email": "customer@example.com"}}
        else:
            return {"tool": "DONE", "parameters": {}}
    
    # Tool mapping
    tool_functions = {
        "search_flights": search_flights,
        "check_seat_availability": check_seat_availability,
        "calculate_total_cost": calculate_total_cost,
        "book_flight": book_flight,
        "send_confirmation": send_confirmation,
    }
    
    iteration = 0
    max_iterations = 10
    
    while iteration < max_iterations:
        iteration += 1
        print(f"\n📍 Iteration {iteration}:")
        
        # Get AI decision
        decision = get_mock_decision(context)
        selected_tool = decision["tool"]
        parameters = decision["parameters"]
        
        print(f"  🤖 AI selected: {selected_tool}")
        
        if selected_tool == "DONE":
            print("  ✅ Goal achieved!")
            break
        
        if selected_tool in tool_functions:
            print(f"  📝 Parameters: {parameters}")
            
            # Execute tool (in Temporal, this would be an Activity)
            result = await tool_functions[selected_tool](parameters)
            
            # Update context
            context += f"\n\nExecuted: {selected_tool}\nResult: {result}"
            
            # Track step
            steps_taken.append(f"{selected_tool}: {result[:80]}...")
            
            print(f"  ✨ Result preview: {result[:100]}...")
    
    print("\n" + "="*60)
    print("📊 EXECUTION COMPLETE\n")
    print("Steps taken:")
    for i, step in enumerate(steps_taken, 1):
        print(f"  {i}. {step}")
    
    return steps_taken

# Run the simulation
steps = await simulate_agentic_loop()

## Step 9: Running with Temporal (Production)

In production, you run this with Temporal for durability. Here's how:

### 1. Start Temporal Server
```bash
temporal server start-dev --ui-port 8080
```

### 2. Run the Worker
```bash
python worker.py
```

### 3. Start a Workflow
```bash
python starter.py
```

Let's look at the worker implementation:

In [ ]:
# Display the worker setup
with open('worker.py', 'r') as f:
    worker_code = f.read()

print("👷 Worker Implementation:\n")
print("```python")
# Show key parts
for line in worker_code.split('\n')[4:40]:  # Skip imports, show main setup
    if line.strip() and not line.strip().startswith('#'):
        print(line)
print("```")

## Step 10: Key Benefits of Temporal for AI Agents

### 🛡️ Durability
- Workflows survive failures and resume from last checkpoint
- No lost progress when LLM calls fail

### 🔄 Automatic Retries
- Activities retry with exponential backoff
- Configurable retry policies per activity

### 📊 Observability
- Full history of all decisions and executions
- Temporal Web UI shows complete workflow state

### 💰 Cost Efficiency
- No repeated LLM calls after failures
- Efficient resource utilization

Let's see how retry policies work:

In [ ]:
from temporalio.common import RetryPolicy
from datetime import timedelta

# Example retry policy for LLM activities
llm_retry_policy = RetryPolicy(
    maximum_attempts=3,
    initial_interval=timedelta(seconds=1),
    maximum_interval=timedelta(seconds=10),
    backoff_coefficient=2.0,
)

print("🔄 Retry Policy for LLM Activities:")
print(f"  • Max attempts: {llm_retry_policy.maximum_attempts}")
print(f"  • Initial interval: {llm_retry_policy.initial_interval}")
print(f"  • Max interval: {llm_retry_policy.maximum_interval}")
print(f"  • Backoff coefficient: {llm_retry_policy.backoff_coefficient}")
print("\n📈 Retry intervals: 1s → 2s → 4s → 8s → 10s (max)")

## Exercise: Extend the Agent

Now it's your turn! Try these exercises:

### Exercise 1: Add a New Tool
Create a new tool for checking weather at the destination:

In [ ]:
# TODO: Implement a weather checking tool
async def check_weather(params):
    """Check weather at destination"""
    destination = params.get("destination", "Unknown")
    # Your implementation here
    pass

# TODO: Create a ToolDefinition for it
weather_tool = ToolDefinition(
    name="check_weather",
    description="Check weather forecast at destination",
    arguments=[
        # Add arguments here
    ]
)

### Exercise 2: Test Error Handling
Modify a tool to simulate failures and see how Temporal handles them:

In [ ]:
# TODO: Create a tool that fails intermittently
import random

async def unreliable_tool(params):
    """Tool that fails 50% of the time"""
    if random.random() < 0.5:
        raise Exception("Network timeout")
    return "Success!"

# Test it
# for i in range(5):
#     try:
#         result = await unreliable_tool({})
#         print(f"Attempt {i+1}: {result}")
#     except Exception as e:
#         print(f"Attempt {i+1}: Failed - {e}")

## Summary

In this module, we've built a durable AI agent using Temporal's agentic loop pattern. Key takeaways:

### 🎯 What We Built
- An autonomous AI agent that books flights
- Dynamic tool selection using LLMs
- Fault-tolerant execution with Temporal

### 🔑 Key Patterns
- **Workflows** orchestrate the agent logic
- **Activities** execute non-deterministic operations
- **Agentic Loop** provides autonomous decision-making
- **Retry Policies** handle transient failures

### 💡 Benefits
- **Durability**: Survives failures and restarts
- **Observability**: Complete audit trail
- **Reliability**: Automatic retries
- **Cost-Effective**: No repeated LLM calls

### 🚀 Next Steps
1. Run the complete system with Temporal
2. Add more sophisticated tools
3. Implement human-in-the-loop with Signals
4. Deploy to production with Temporal Cloud

**Remember**: Temporal transforms unreliable AI agents into robust, production-ready systems!

## Resources

- 📚 [Temporal AI Agent Tutorial](https://learn.temporal.io/tutorials/ai/durable-ai-agent/)
- 🐍 [Temporal Python SDK](https://docs.temporal.io/develop/python)
- 🤖 [LiteLLM Documentation](https://docs.litellm.ai/)
- 🔧 [OpenAI Function Calling](https://platform.openai.com/docs/guides/function-calling)
- 🎥 [Temporal YouTube Channel](https://www.youtube.com/@Temporalio)

**Happy building with Temporal! 🚀**